In [ ]:
import pandas as pd
import blimpy as bl
import setigen as stg
import numpy as np
from concurrent.futures import ProcessPoolExecutor

# --- SETTINGS ---
csv_path = 'your_files.csv'
n_coarse = 16
coarse_width = 1024
max_workers = 4  # Tune this for your server

# --- LOAD CSV ---
df = pd.read_csv(csv_path)

# --- PREPARE ALL TASKS ---
tasks = []
for _, row in df.iterrows():
    h5_path = row['.h5 path']
    for coarse_idx in range(n_coarse):
        tasks.append((h5_path, coarse_idx))

# --- DEFINE PROCESSING FUNCTION ---
def process_coarse_channel(args):
    h5_path, coarse_idx = args
    try:
        fb = bl.Waterfall(h5_path)
        data = fb.data.squeeze()
        start = coarse_idx * coarse_width
        end = (coarse_idx + 1) * coarse_width
        coarse_data = data[:, start:end]
        frame = stg.Frame(coarse_data)
        # --- Injection logic goes here ---
        # frame.add_noise(...)
        # frame.add_signal(...)
        # Save or return results
        print(f"Done: {h5_path}, coarse channel {coarse_idx}")
        return (h5_path, coarse_idx, True)  # Or whatever result you need
    except Exception as e:
        print(f"Error processing {h5_path}, coarse {coarse_idx}: {e}")
        return (h5_path, coarse_idx, False)

# --- RUN PARALLEL PROCESSING ---
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    results = list(executor.map(process_coarse_channel, tasks))

print("All jobs completed.")
